In [ ]:
!python --version

Python 3.10.12


In [ ]:
import gc
import tensorflow as tf

gc.collect()

2689

# Installing required Libraries

In [ ]:
!pip install tabula-py
!pip install pypdf
!pip install PyPDF2
!pip install fuzzywuzzy
!pip install frontend
!pip install wget
!pip install pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=316b49b88290b9f1df91397f15d2e12c288ca3188b8f240644

In [ ]:
!python3 -m pip install "pypdfium2==4" pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.3 MB/s eta 0:00:00


In [ ]:
#!pip install -q transformers einops accelerate langchain bitsandbytes sentence_transformers llama-index pypdf python-dotenv


# Importing required Libraries

In [ ]:

import requests
from bs4 import BeautifulSoup
from lxml import etree
from fuzzywuzzy import fuzz
import PyPDF2
import wget
from google.colab import files
import tabula
import numpy as np
from IPython.display import display,Image
from pdf2image import convert_from_path
import pypdfium2 as pdfium



/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Functions to parse the DOM

In [ ]:
def get_dom_object(url,HEADERS):

  response = requests.get(url,HEADERS)
  #print(url,response.status_code )
  soup = BeautifulSoup(response.content, "html.parser")
  dom = etree.HTML(str(soup))

  return dom,(response.status_code == 200)

In [ ]:
def get_data_with_xpath(dom,xpath,company_code,base_url):

  part=None
  report_url=""
  is_sumplimental=False

  if(company_code=="MGS"):
    element_list=dom.xpath(xpath)
    if(element_list):
      element=element_list[0]
      part=element.attrib["href"]
      report_url=base_url+part
      #print(report_url)
      return report_url,(part!=None)

  if(company_code=="BOFA"):
    element_list=dom.xpath(xpath)

    if(element_list):
      element=element_list[0]
      report_url=element.attrib["href"]
      #print(etree.tostring(element))
      text='random'
      try:
        text=element.attrib["aria-label"]
      except:
        text='random'

      is_sumplimental =True if fuzz.partial_ratio(text.lower(),"supplemental information")>=90 else False


      return report_url,is_sumplimental



In [ ]:
def get_company_data(dom,comany,current_year):

  data={}
  if(comany["Code"]=="MGS"):
    for year in range(comany["report_avalaible_from"],comany["report_avalaible_till"]-1,-1):
        data[str(year)]=[]
        for quater in range(2,6):
          obj={}
          obj["Quarter"]=quater-1
          obj["FinancialSuppliments"]=""
          obj["isAvailabe"]=False
          index=current_year-year
          xpath_morgan_stanley=f"//html//body//div[1]//div[2]//div//div//div//section//article//table//tbody//tr[{index}]//td[{quater}]//a[2]"
          #print(xpath_morgan_stanley)

          report_url,is_available=get_data_with_xpath(dom,xpath_morgan_stanley,comany["Code"],comany["base_url"])
          if(is_available):
            obj["Financial Suppliments"]=report_url
            obj["isAvailabe"]=True
          data[str(year)].append(obj)

  if(comany["Code"]=="BOFA"):
    xpath_bofa=f"//*[@id='main']/div/div"
    element_list=dom.xpath(xpath_bofa)
    id_list=[ele.attrib["id"] for ele in element_list]
    for id in id_list:
      year=id.split("-")[0]
      data[year]=[]
      for quater in range(1,5):
        for order in range(3,6):
          obj={}
          obj["Quarter"]=4-quater+1
          obj["FinancialSuppliments"]=""
          obj["isAvailabe"]=False
          xpath_bofa_anchor=f"//*[@id='{id}-box']/div/div[{quater}]/div/div/div[2]/div[{order}]/div/a"
          report_url,is_available=get_data_with_xpath(dom,xpath_bofa_anchor,comany["Code"],comany["base_url"])
          if(is_available):
              obj["FinancialSuppliments"]=report_url
              obj["isAvailabe"]=True
              data[year].append(obj)
              break
              #print(id,quater,report_url)

  return data

# Companies List to retreive data

In [ ]:
company_list=[

             {   "Name":"Morgan Stanley",
                 "Code":"MGS",
                 "report_avalaible_from":2023,
                 "report_avalaible_till":2002,
                 "url":"https://www.morganstanley.com/about-us-ir/earnings-releases",
                 "base_url":"https://www.morganstanley.com"
             },
              {  "Name":"Bank of America",
                 "Code":"BOFA",
                 "report_avalaible_from":2023,
                 "report_avalaible_till":2018,
                 "url":"https://investor.bankofamerica.com/quarterly-earnings",
                 "base_url":"https://investor.bankofamerica.com/quarterly-earnings"
             }

]
# gs bofa
current_year=2024

HEADERS = ({'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \ (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})


In [ ]:

all_comany_pdfs=[]
data=[]
for comany in company_list[1:]:
  dom,is_success=get_dom_object(comany["url"],HEADERS)
  if(is_success):
    data=get_company_data(dom,comany,current_year)

  all_comany_pdfs.append({comany["Code"] : data })





In [ ]:
all_comany_pdfs[0]['BOFA']['2023']

[{'Quarter': 4,
  'FinancialSuppliments': 'https://d1io3yog0oux5.cloudfront.net/_f20d81f221f7509edf403dc45e57d072/bankofamerica/db/806/9995/supplemental_information/The+Supplemental+Information_4Q23_ADA.pdf',
  'isAvailabe': True},
 {'Quarter': 3,
  'FinancialSuppliments': 'https://d1io3yog0oux5.cloudfront.net/_f20d81f221f7509edf403dc45e57d072/bankofamerica/db/806/9961/supplemental_information/The+Supplemental+Information_3Q23_ADA.pdf',
  'isAvailabe': True},
 {'Quarter': 2,
  'FinancialSuppliments': 'https://d1io3yog0oux5.cloudfront.net/_f20d81f221f7509edf403dc45e57d072/bankofamerica/db/806/9921/supplemental_information/The+Supplemental+Information_2Q23_ADA.pdf',
  'isAvailabe': True},
 {'Quarter': 1,
  'FinancialSuppliments': 'https://d1io3yog0oux5.cloudfront.net/_f20d81f221f7509edf403dc45e57d072/bankofamerica/db/806/9848/supplemental_information/The+Supplemental+Information_1Q23_ADA.pdf',
  'isAvailabe': True}]

#Download the Financial Suppliments Pdf


In [ ]:

import requests

def download_pdf(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as pdf_file:
        pdf_file.write(response.content)

In [ ]:
all_pdf=[]
for year in range(2023,2022,-1):
  for quater_data in  all_comany_pdfs[0]['BOFA'][str(year)]:
    pdf_url=quater_data['FinancialSuppliments']
    local_path=f"/content/BOFA_{year}_{quater_data['Quarter']}.pdf"
    try:
      download_pdf(pdf_url, local_path)
    except:
      print("Error")

    all_pdf.append(local_path)






In [ ]:
all_pdf

['/content/BOFA_2023_4.pdf',
 '/content/BOFA_2023_3.pdf',
 '/content/BOFA_2023_2.pdf',
 '/content/BOFA_2023_1.pdf']

#Extract Pages that has tables related to Wealth Management



In [103]:

def extract_pages_with_text(pdf_path, search_texts, threshold=80):
    matching_pages = []
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
          page = pdf_reader.pages[page_num]
          page_text = page.extract_text()

          similarity = 0
          for search_text in search_texts:
            similarity+=fuzz.partial_ratio(search_text.lower(), page_text.lower())

          if similarity >= threshold*len(search_texts):
              matching_pages.append(page_num + 1)

    return matching_pages

In [104]:


search_texts = ['Wealth Management','Income Statement Information','Financial Information and Statistical Data']

search_texts_GS = ['Assets Under Supervision','AUS']

search_texts_BOFA = ['Assets under management','Global Wealth & Investment','Key Indicators']


matching_pages = extract_pages_with_text(all_pdf[-1], search_texts_BOFA,threshold=60)

if matching_pages:
    print(f"Pages with '{search_texts_BOFA}': {matching_pages}")
else:
    print(f"No pages found with '{search_texts_BOFA}'")


Pages with '['Assets under management', 'Global Wealth & Investment', 'Key Indicators']': [15, 18]


In [135]:
index=-1

In [136]:
all_pdf[index]

'/content/BOFA_2023_1.pdf'

#Extract Required Tables from Pdf

In [137]:
def remove_unnecesary_cols(tables,threshold):
  tables_new=[]
  for i in range(len(tables)):
    data_frame=tables[i]
    data_frame=data_frame.replace('$', np.nan)
    data_frame=data_frame.dropna(thresh=len(data_frame)-threshold, axis=1)
    tables_new.append(data_frame)
  return tables_new



In [138]:
pdf = pdfium.PdfDocument(all_pdf[index])
images=[]
for i in matching_pages:
    page = pdf[i-1]
    image = page.render(scale=4).to_pil()
    images.append(image)
    image.save(f"output_bofa_{i-1:03d}.jpg")

In [139]:
from transformers import DetrFeatureExtractor

feature_extractor = DetrFeatureExtractor()
encoding = feature_extractor(images[0], return_tensors="pt")
encoding.keys()

/usr/local/lib/python3.10/dist-packages/transformers/models/detr/feature_extraction_detr.py:38: FutureWarning: The class DetrFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DetrImageProcessor instead.
  warnings.warn(


dict_keys(['pixel_values', 'pixel_mask'])

In [140]:
tables = tabula.read_pdf(all_pdf[index],columns=[57, 370, 430, 460, 520, 580],multiple_tables=True,pages=matching_pages)
tables_new=[]
for i, df in enumerate(tables):
    df=remove_unnecesary_cols([df],len(df)-3)[0]
    tables_new.append(df)
    display(df)


,(Dollars in millions),Unnamed: 1,Unnamed: 3,Unnamed: 5,Unnamed: 7,Unnamed: 9
0,NaN,First,Fourth,Third,Second,First
1,NaN,Quarter,Quarter,Quarter,Quarter,Quarter
2,NaN,2023,2022,2022,2022,2022
3,Average deposit balances,NaN,NaN,NaN,NaN,NaN
4,Checking,"580,910","588,668","599,099","606,331","593,428"
5,Savings,"68,327","69,790","71,933","73,295","72,413"
6,MMS,"339,823","356,015","365,271","362,798","354,850"
7,CDs and IRAs,"33,098","28,619","28,731","29,796","30,685"
8,Other,"4,084","3,966","4,059","5,800","4,724"
9,Total average deposit balances,NaN,"1,047,058","1,069,093","1,078,020","1,056,100"


,(Dollars in millions),Unnamed: 1,Unnamed: 3,Unnamed: 5,Unnamed: 7,Unnamed: 9
0,NaN,First,Fourth,Third,Second,First
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Quarter,Quarter,Quarter,Quarter,Quarter
3,NaN,2023,2022,2022,2022,2022
4,Revenue by Business,NaN,NaN,NaN,NaN,NaN
5,Merrill Wealth Management,"4,397","4,486","4,524","4,536","4,589"
6,Bank of America Private Bank,918,924,905,897,887
7,"Total revenue, net of interest expense","5,315","5,410","5,429","5,433","5,476"
8,"Client Balances by Business, at period end",NaN,NaN,NaN,NaN,NaN
9,Merrill Wealth Management,"2,952,681","2,822,910","2,710,985","2,819,998","3,116,052"


In [141]:
def fuzzy_match(row, target):
    threshold=90
    return (fuzz.partial_ratio(str(row.iloc[0]), target) >= threshold)



In [142]:
map={
    "Merrill Wealth Management": {
        "count": 2,
        "Keys": ["Merrill WM Rev", "Merrill Client Balances"],
        "Value":[]
    },
    "Bank of America Private Bank": {
        "count": 2,
        "Keys": ["Private Bank Rev", "Private Bank Balances"],
        "Value":[]
    },
    "Total client balances": {"count": 1, "Keys": ["Total Client Balances"],"Value":[]},
    "Total assets under management, ending": {"count": 1, "Keys": ["Wealth AUM"],"Value":[]},
    "Net client flows": {"count": 1, "Keys": ["Wealth Net Flows"],"Value":[]},
}

for key in map.keys():
  for table in tables_new:
    result = table.apply(fuzzy_match, args=(key,),axis=1)
    df=table[result]
    for i in range(len(df)):
      map[key]["Value"].append(df.iloc[i,1])
      if(map[key]["count"]==1):
        break








In [143]:
map

{'Merrill Wealth Management': {'count': 2, 'Keys': ['Merrill WM Rev', 'Merrill Client Balances'], 'Value': ['4,397', '2,952,681']}, 'Bank of America Private Bank': {'count': 2, 'Keys': ['Private Bank Rev', 'Private Bank Balances'], 'Value': ['918', '568,925']}, 'Total client balances': {'count': 1, 'Keys': ['Total Client Balances'], 'Value': ['3,521,606']}, 'Total assets under management, ending': {'count': 1, 'Keys': ['Wealth AUM'], 'Value': ['1,467,242']}, 'Net client flows': {'count': 1, 'Keys': ['Wealth Net Flows'], 'Value': ['15,262']}}

#Split dataframes from tables

In [ ]:
def split_data_frame(data_frame,nan_threshold):
  nan_rows = data_frame[data_frame.isna().sum(axis=1) >= nan_threshold]
  column_to_extract='Unnamed: 0'
  column_data = nan_rows[column_to_extract]
  split_dataframes = np.split(data_frame, nan_rows.index)
  return split_dataframes,column_data


In [ ]:
splited_dataframes,column_data=split_data_frame(tables[2],tables[2].shape[1]-2)

In [ ]:
len(column_data)
column_data=list(column_data)

In [ ]:
len(splited_dataframes)

4

In [ ]:
for i, df in enumerate(splited_dataframes):

    display(df)

,Unnamed: 0,"Sep 30, 2022","Jun 30, 2022","Sep 30, 2021","Jun 30, 2022.1","Sep 30, 2021.1","Sep 30, 2022.1","Sep 30, 2021.2",Change


,Unnamed: 0,"Sep 30, 2022","Jun 30, 2022","Sep 30, 2021","Jun 30, 2022.1","Sep 30, 2021.1","Sep 30, 2022.1","Sep 30, 2021.2",Change
0,Revenues:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Asset management,"$3,389","$3,510","$3,628",(3%),(7%),"$10,525","$10,266",3%
2,Transactional,616,291,832,112%,(26%),"1,542","3,232",(52%)
3,Net interest income,"2,004","1,747","1,348",15%,49%,"5,291","3,988",33%
4,Other,111,188,127,(41%),(13%),433,503,(14%)
5,Net revenues,"6,120","5,736","5,935",7%,3%,"17,791","17,989",(1%)
6,Provision for credit losses,11,19,‐,(42%),*,43,(2),*
7,Compensation and benefits,"3,171","2,895","3,159",10%,‐‐,"9,191","9,604",(4%)
8,Non‐compensation expenses,"1,289","1,301","1,246",(1%),3%,"3,814","3,621",5%


,Unnamed: 0,"Sep 30, 2022","Jun 30, 2022","Sep 30, 2021","Jun 30, 2022.1","Sep 30, 2021.1","Sep 30, 2022.1","Sep 30, 2021.2",Change
9,(1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Total non‐interest expenses,"4,460","4,196","4,405",6%,1%,"13,005","13,225",(2%)
11,Income before provision for income taxes,"1,649","1,521","1,530",8%,8%,"4,743","4,766",‐‐
12,Net income applicable to Morgan Stanley,"$1,253","$1,190","$1,157",5%,8%,"$3,715","$3,663",1%
13,Pre‐tax profit margin,27%,27%,26%,NaN,NaN,27%,26%,NaN
14,Pre‐tax profit margin excluding integration‐re...,28%,28%,28%,NaN,NaN,28%,28%,NaN
15,Compensation and benefits as a % of net revenues,52%,50%,53%,NaN,NaN,52%,53%,NaN
16,Non‐compensation expenses as a % of net revenues,21%,23%,21%,NaN,NaN,21%,20%,NaN
17,Return on Average Common Equity,16%,15%,16%,NaN,NaN,16%,17%,NaN


,Unnamed: 0,"Sep 30, 2022","Jun 30, 2022","Sep 30, 2021","Jun 30, 2022.1","Sep 30, 2021.1","Sep 30, 2022.1","Sep 30, 2021.2",Change
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,Return on Average Tangible Common Equity,30%,29%,34%,NaN,NaN,30%,35%,NaN


In [ ]:
data_to_llm=[]

for i, df in enumerate(splited_dataframes):
  if(i==0):
    continue
  data_str=""

  df=df.iloc[1:]
  print(f"Table  {column_data[i-1]} ")
  data_str+=f"Table  {column_data[i-1]} "

  data_str+="\n"

  print()
  df_array=remove_unnecesary_cols([df],1)

  print(df_array[0].to_markdown())
  data_str+=df_array[0].to_markdown()
  data_str+="\n\n"
  print()
  print()
  print()
  data_to_llm.append(data_str)


Table  Revenues: 

|    | Unnamed: 0                  | Sep 30, 2022   | Jun 30, 2022   | Sep 30, 2021   | Jun 30, 2022.1   | Sep 30, 2021.1   | Sep 30, 2022.1   | Sep 30, 2021.2   | Change   |
|---:|:----------------------------|:---------------|:---------------|:---------------|:-----------------|:-----------------|:-----------------|:-----------------|:---------|
|  1 | Asset management            | $3,389         | $3,510         | $3,628         | (3%)             | (7%)             | $10,525          | $10,266          | 3%       |
|  2 | Transactional               | 616            | 291            | 832            | 112%             | (26%)            | 1,542            | 3,232            | (52%)    |
|  3 | Net interest income         | 2,004          | 1,747          | 1,348          | 15%              | 49%              | 5,291            | 3,988            | 33%      |
|  4 | Other                       | 111            | 188            | 127            | (41%)            |

#Use LLM TO extract any data from the tables


In [ ]:
!pip install transformers
!pip install accelerate

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) N
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


NameError: name 'data_to_llm' is not defined

In [ ]:
user_queries = ['what is the value of fee based client assets during march 31, 2023 ?' , 'Self directed assets in Mar 31, 2023 ?' ]

all_data="\n".join(data_to_llm)
all_queries=" and  ".join(user_queries)

final_question=all_data+all_queries


In [ ]:
d=f''

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)
sequence = pipeline(
    f'I have this table "{d}". Do you {user_queries[0]} from the table \n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

for seq in sequence:
   print(f"Result: {seq['generated_text']}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Comment: Of course! Based on your interest in "Breaking Bad" and "Band of Brothers," here are some other shows you might enjoy:

1. "The Wire" - A gritty and intense drama that explores the drug trade in Baltimore from multiple perspectives.
2. "Sons of Anarchy" - A biker drama that follows the lives of a motorcycle club involved in illegal activities.
3. "The Sopranos" - A classic HBO drama that explores the life of a New Jersey mob boss and his family.
4. "Narcos" - A Netflix series that tells the true story of Pablo Escobar and the Medellín cartel.
5. "Peaky Blinders" - A historical crime drama


In [ ]:
import logging
import sys
import torch
from pprint import pprint
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.readers import StringIterableReader
from llama_index.llms import HuggingFaceLLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding
from llama_index import ServiceContext
from llama_index.prompts.prompts import SimpleInputPrompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
!git config --global credential.helper store
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.ca

In [ ]:
# hyperparameters
context_window = 4096 #@param
temperature = 0.0 #@param
model_name = 'meta-llama/Llama-2-13b-chat-hf' #@param

llm = HuggingFaceLLM(
    context_window=context_window,
    max_new_tokens=256,
    generate_kwargs={"temperature": temperature, "do_sample": False},
    tokenizer_name=model_name,
    model_name=model_name,
    device_map="auto",
   #uncomment this if using CUDA to reduce memory usage
    #model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True, "use_auth_token":True},

)
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the disk and cpu.


In [ ]:
print(data_to_llm[2])

Table  dvisor-led channel 
|    | Unnamed: 0                                           | Mar 31, 2023   | Dec 31, 2022   | Mar 31, 2022   | Dec 31, 2022.1   | Mar 31, 2022.1   |
|---:|:-----------------------------------------------------|:---------------|:---------------|:---------------|:-----------------|:-----------------|
| 10 | Advisor-led client assets                            | 3,582          | 3,392          | 3,835          | 6%               | (7%)             |
| 11 | Fee-based client assets                              | 1,769          | 1,678          | 1,873          | 5%               | (6%)             |
| 12 | Fee-based asset flows                                | 22.4           | 20.4           | 97.2           | 10%              | (77%)            |
| 13 | Fee-based assets as a % of advisor-led client assets | 49%            | 49%            | 49%            | nan              | nan              |




In [ ]:
documents = StringIterableReader().load_data(
    texts=data_to_llm
)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)


In [ ]:
user_queries = ['what is the value of fee based client assets during march 31, 2023 ?' , 'Self directed assets in Mar 31, 2023 ' ]
answer = dict()
for i, user_query in enumerate(user_queries):
    query_engine = index.as_query_engine()
    torch.cuda.empty_cache()
    response = query_engine.query(user_query)
    answer.update({user_query: response.response})
pprint(answer)

OutOfMemoryError: CUDA out of memory. Tried to allocate 100.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 99.06 MiB is free. Process 33507 has 14.65 GiB memory in use. Of the allocated memory 14.22 GiB is allocated by PyTorch, and 315.15 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
answer

In [ ]:
import torch

